# Preprocessing, GridSearch and Training CNN-3, CNN-6 and CNN-8 on Private Data

In [1]:
import numpy as np
from numpy import array
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import math

import matplotlib.pylab as plt
from matplotlib.pyplot import cm
import seaborn as sns
import itertools
import collections
from collections import OrderedDict, Counter

import scipy.stats as stats
from scipy.stats import normaltest, kstest, ttest_ind, ttest_rel, mannwhitneyu, wilcoxon, levene, bartlett
from scipy.stats import chi2_contingency, fisher_exact, mode, pearsonr, f_oneway, kruskal, spearmanr

from rfpimp import *

from datetime import datetime
import re
from seaborn import heatmap
import random
import statsmodels.distributions.empirical_distribution as edf
from scipy.interpolate import interp1d
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.utils import resample
from functools import reduce
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, log_loss, recall_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, PowerTransformer

from sklearn.utils import check_consistent_length, column_or_1d, assert_all_finite
from sklearn.utils.extmath import stable_cumsum
from sklearn.utils.multiclass import type_of_target
from sklearn.linear_model import LogisticRegression
import re
import gc

import os

#from sklearn.externals import joblib
import joblib

from importlib import reload
import Filter_and_Grid_Search
Filter_and_Grid_Search = reload(Filter_and_Grid_Search)
from Filter_and_Grid_Search import stratified_split
from Filter_and_Grid_Search import attributes_list, attributes_list_new
from Filter_and_Grid_Search import get_s_stat, get_PSI_stat, get_stats_by_month, get_stats, stable_unstable
from Filter_and_Grid_Search import stable_unstable_by_month_divide, union_datas, individual_hists_all 
from Filter_and_Grid_Search import paired_time_hists_by_month, statistics_with_target
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import statistics_with_target, attributes_list, attributes_list_new, make_standard
from Filter_and_Grid_Search import data_preprocessing_train, data_preprocessing_test
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import stratified_split, two_forests, turn_variables_with_values
from Filter_and_Grid_Search import find_meta_params, calculate_vif#, find_meta_params_mem
from Filter_and_Grid_Search import plot_meta_2d, data_preprocessing, find_ouliers_iqr
from Filter_and_Grid_Search import train_model_receive_stats, simple_b_score_risk
from Filter_and_Grid_Search import max_prof_corve, by_month_gini, check_attribute_list_cases

from Filter_and_Grid_Search import to_zip, br_correction, br_stat

from statsmodels.stats.proportion import proportions_ztest

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pathlib
import psutil

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
PATH_DATA = 'D:/Share/01 -Risk Desc Science/AntiFraud/Data Sets/'
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_v2_2014/'
print(PATH_DATA)
print(PATH)

D:/Share/01 -Risk Desc Science/AntiFraud/Data Sets/
D:/Share/safanasev/Python-notebook/AF_ML_v2_2014/


In [3]:
train_data = 'antifraud_data_train_with_2014.csv'
test_data  = 'antifraud_data_test_with_2014.csv'
val_data   = 'antifraud_data_valid_with_2014.csv'

In [4]:
task = 'binary'
missing_strings = 'MISSING'
p_value = 0.05
target_dict = {'good': 0, 'bad': 1}

In [5]:
target = 'BAD_FLAG'
index_month = 'MONTH_YEAR'
list_of_vars_for_strat = ['MONTH_YEAR']
sort_by_var = 'APPPOSID'

necessary_fields = [target, index_month, sort_by_var]

COL_DEL = ['Unnamed: 0', 'PERIOD_7', 'LOSS_90P'] 
COL_DEL = [x.upper() for x in COL_DEL]
COL_TRG = target

In [6]:
n_jobs = 3

In [7]:
path_train_data = PATH_DATA  +  train_data

In [8]:
#new_features_data_no_vif = pd.read_csv(PATH + '/SandBox/selected_features_forest.csv', sep = '\t', index_col=0)
#new_features_data_no_vif_no_sys = pd.read_csv(PATH + '/SandBox/selected_features_forest_vif_no_sys.csv', sep = '\t', index_col = 0)
#categorials_data_fin = pd.read_csv(PATH + 'SandBox/categories_fin.csv', sep = '\t', index_col=0)
#definition_2 = pd.read_csv(PATH + 'SandBox/definition_2.csv', index_col=0)

In [132]:
PATH2 = 'D:/Share/01 -Risk Desc Science/AntiFraud/Selection_new/SandBox/'

new_features_data_no_vif = pd.read_csv(PATH2 + 'selected_features_forest_vif1.csv', sep = '&', index_col=0)
new_features_data_no_vif_no_sys = pd.read_csv(PATH2 + 'selected_features_forest_vif_no_sys1.csv', sep = '&', index_col=0)
categorials_data_fin = pd.read_csv(PATH2 + 'categories_fin.csv', sep = '&', index_col=0)

In [133]:
selected_features = new_features_data_no_vif

In [134]:
cols = selected_features['genuine variable'].drop_duplicates().to_list()

In [135]:
# Загружаем только необходимые колонки и в нужном формате
# 'cp1251'
raw_columns = (pd.read_csv(PATH_DATA +  train_data, sep = '&', encoding = 'utf-8', nrows = 0)).columns # исходные имена колонок
#dtype={c:type_cols.get(c.upper(),'float32') for c in raw_columns}                                                  # справочник для исходных колонок
train = pd.read_csv(PATH_DATA  +  train_data, sep = '&', encoding = 'utf-8',  #dtype=dtype, 
                    usecols=[c for c in raw_columns if c.upper() in cols])
test  = pd.read_csv(PATH_DATA +  test_data,  sep = '&', encoding = 'utf-8',  
                    #dtype=dtype, 
                    usecols=[c for c in raw_columns if c.upper() in cols])
val   = pd.read_csv(PATH_DATA +   val_data,   sep = '&', encoding = 'cp1251',  
                    #dtype=dtype, 
                    usecols=[c for c in raw_columns if c.upper() in cols])
train.columns = [col.upper() for col in train.columns]
test.columns  = [col.upper() for col in test.columns]
val.columns   = [col.upper() for col in val.columns]

In [136]:
train = turn_variables_with_values(train, selected_features)
test  = turn_variables_with_values(test,  selected_features)
val   = turn_variables_with_values(val,   selected_features)

In [137]:
y_train = train[target]
y_test = test[target]
y_val = val[target]

train.drop(target, axis = 1, inplace = True)
test.drop(target, axis = 1, inplace = True)
val.drop(target, axis = 1, inplace = True)

In [138]:
y_train = y_train.astype('int')
y_test  = y_test.astype('int')
y_val   = y_val.astype('int')

In [139]:
list_of_columns = selected_features['new variable'].to_list()
list_of_columns_no_sys = [c for c in list_of_columns if c not in necessary_fields]

In [140]:
attribute_list_model = attributes_list_new(train, list_of_columns_no_sys, [0.01, 0.5, 1, 2, 5, 95, 97, 
                                                                           98, 99, 99.5, 99.99])

[1.000e-04 5.000e-03 1.000e-02 2.000e-02 5.000e-02 9.500e-01 9.700e-01
 9.800e-01 9.900e-01 9.950e-01 9.999e-01]
Number of finished repetitions: 0 | time:  23.09.2020 18:26:48


In [141]:
list_of_categories_sel = attribute_list_model.loc[(attribute_list_model['count_dist'] == 2), 'attribute'].to_list()

In [142]:
for i in list_of_columns_no_sys:
    if i in categorials_data_fin:
        list_of_categories_sel.append(i)

In [143]:
attribute_list_model['right_border'] = attribute_list_model['99%']
attribute_list_model['left_border'] = attribute_list_model['1%']

In [8]:
from importlib import reload
import Filter_and_Grid_Search
Filter_and_Grid_Search = reload(Filter_and_Grid_Search)
from Filter_and_Grid_Search import data_preprocessing_train

In [145]:
st_of_columns_no_sys = [c for c in list_of_columns if c not in necessary_fields]
cols_outliers = list_of_columns_no_sys.copy()

In [146]:
X_1_2,  preproc_ = data_preprocessing_train(train, y_train, #test, y_test, 
                                                    technical_values = necessary_fields, 
                                                       categorial_list = list_of_categories_sel, drop_technical = True,
                                                       yeo_johnson = False, attribute_list = attribute_list_model, 
                                                       var_col = 'attribute',
                                                       scale = 'mean', 
                                                       median = 'val_mediana', 
                                                       
                                                       high_outlier = 'right_border', 
                                                       low_outlier = 'left_border', 
                                                    check_percentile = 5,
                                                       cols_outlier = cols_outliers, 
                                                       cut_non_out_9999 = True)

In [147]:
X_2_2 = data_preprocessing_test(test, y_test, necessary_fields, categorial_list = list_of_categories_sel, 
                                drop_technical = True,
                                attribute_list = attribute_list_model, var_col = 'attribute',
                                median = 'val_mediana',
                                high_outlier = 'right_border',
                                low_outlier = 'left_border', 
                                scale =  preproc_, 
                                #yeo_johnson = None,
                                cols_outlier = cols_outliers)

In [148]:
X_3_2 = data_preprocessing_test(val, y_val, necessary_fields, categorial_list = list_of_categories_sel, 
                                drop_technical = True,
                                attribute_list = attribute_list_model, var_col = 'attribute',
                                median = 'val_mediana',
                                high_outlier = 'right_border',
                                low_outlier = 'left_border', 
                                scale =  preproc_, 
                                #yeo_johnson = None,
                                cols_outlier = cols_outliers)

# Tensor

In [12]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 29

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)


# 5. Configure a new global `tensorflow` session
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Activation, Dense, Flatten, Dropout)
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, AveragePooling1D)
from tensorflow.keras.layers import add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from keras.utils import np_utils

In [15]:
from tensorflow.keras.layers import ReLU, Concatenate, GlobalAveragePooling1D

In [16]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11769004741369248502
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16846372820444193308
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4908410703065105795
physical_device_desc: "device: XLA_GPU device"
]


In [21]:
#если alpha не задано, то не делаем br_correction

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,  dataset, x_col, y_col,  batch_size=256, num_classes=2, alpha = 0.1, random_state = 42, 
                 shuffle=True, class_w = None):
        self.batch_size = batch_size
        self.dataset = dataset
        self.x_col = x_col
        self.y_col = y_col
        self.indices = self.dataset.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.alpha = alpha
        self.random_state = random_state
        self.class_w = class_w
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.indices) / self.batch_size)

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        data_tmp = self.dataset.loc[batch]
        data_tmp_b = data_tmp[data_tmp[self.y_col] == 1]
        data_tmp_b_all = self.dataset[self.dataset[self.y_col] == 1]
        X_tmp = data_tmp.head(0)
        
        
        #print(data_tmp_b.shape[0], data_tmp.shape[0], data_tmp_b_all.shape[0])
        if (self.alpha is None) and (self.class_w is None):
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col] 
            
        elif (self.alpha is None) and self.class_w > 0:
            k = (self.class_w*self.batch_size)/(1-self.class_w)
            k = k - data_tmp_b.shape[0]
            if int(k) < 1:
                k = k+1
            ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
            X_tmp = data_tmp_b_all.iloc[ix2]
            data_tmp = data_tmp.append(X_tmp, ignore_index=True)
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]
            #print(data_tmp_b.shape[0], data_tmp.shape[0],  data_tmp_b.shape[0]/(data_tmp.shape[0] - X_tmp.shape[0]), np.mean(y)) 
            
            
            
        else:
            if data_tmp_b.shape[0] > 0:
                k=(self.alpha*self.batch_size/data_tmp_b.shape[0] -1 )/(1-self.alpha)
                for i in range(0,int(k)):
                    X_tmp = X_tmp.append(data_tmp_b, ignore_index=True)
            else:
                k = (self.alpha*self.batch_size)/(1-self.alpha)
                ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
                X_tmp = data_tmp_b_all.iloc[ix2]
                data_tmp_b = X_tmp

    #         print('Добавим дробное число строк')    
            k_fraction = k - int(k)
            n_samples = int(round(data_tmp_b.shape[0]*k_fraction))
    #         print(k_fraction, n_samples)

            ix = np.random.RandomState(self.random_state).choice(data_tmp_b.shape[0], n_samples)
            data_add_fraction = data_tmp_b.iloc[ix]
            X_tmp.append(data_add_fraction, ignore_index=True)

            data_tmp = data_tmp.append(X_tmp, ignore_index=True)


            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]   
        #print(sum(self.dataset.loc[batch][self.y_col])/len(index), np.mean(y))
        
        if self.num_classes > 1:
            y = np_utils.to_categorical( y, self.num_classes)
            
        ####for tf 2.3.0, [np.array(X)], np.array(y)

        #return [np.array(X).reshape(X.shape[0], X.shape[1], 1)], np.array(y)
        return [np.array(X)], np.array(y)

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping, Callback

# объявляем класс метрик

class E_time(Callback):
    def on_train_begin(self, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('train_begin', '| time: ' , tm)
        
    def on_epoch_end(self, epoch, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('epoch_end', '| time: ' , tm)
    
        return

_time = E_time()

In [19]:
class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, valid_data):
        super(Metrics, self).__init__()
        self.validation_data = valid_data

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_predict = self.model.predict(self.validation_data[0])[:, 1]
        val_targ = self.validation_data[1]
        
        if len(val_targ.shape) == 2: #and val_targ.shape[1] != 1:
            val_targ = val_targ[:,1]

        _val_aps = metrics.average_precision_score(val_targ, val_predict)
        #_val_recall = metrics.recall_score(val_targ, val_predict)
        _val_a = metrics.roc_auc_score(val_targ, val_predict)

        logs['val_aps'] = _val_aps
        logs['val_a'] = _val_a
        print(" — val_aps:  %f — val_a: %f" % (_val_aps, _val_a))
        return

## Loading preprocessed datasets

In [ ]:
#PATH + 'train_51_prep.csv'
#PATH + 'train_163_prep.csv'

In [8]:
train_for = pd.read_csv(PATH + 'train_163_prep.csv')
valid_for = pd.read_csv(PATH + 'valid_163_prep.csv')
test_for = pd.read_csv(PATH + 'test_163_prep.csv')

In [9]:
y_train = train_for[target]
y_test = valid_for[target]
y_val = test_for[target]

train_for.drop('Unnamed: 0', axis = 1, inplace = True)
valid_for.drop('Unnamed: 0', axis = 1, inplace = True)
test_for.drop('Unnamed: 0', axis = 1, inplace = True)


col = train_for.columns.to_list()
col.remove(target)


X_1_2 = train_for[col]
X_2_2 = valid_for[col]
X_3_2 = test_for[col]

In [10]:
X_2_2.columns

Index(['COUNT_CONT_30', 'W_TEST_AGE_6_60D', 'W_TEST_AGE_ALL5_7D',
       'B_TEST_PASP_D_3_7D', 'B_TEST_PASP_D_3_14D', 'B_TEST_PASP_D_3_30D',
       'B_TEST_PASP_D_3_60D', 'B_TEST_PASP_D_10_7D', 'B_TEST_PASP_D_10_30D',
       'B_TEST_PASP_D_ALL5_30D',
       ...
       'DIVISION_Южный', 'AVG_FUND_TT_7_bin', 'AVG_FUND_TT_14_bin',
       'STD_SUM_TT_14_bin', 'AVG_FUND_TT_30_bin', 'STD_SUM_TT_30_bin',
       'STD_SUM_TT_60_bin', 'RULE_1_TRE3_SHARE_7_bin',
       'RULE_1_TRE3_SHARE_14_bin', 'COUNT_CONT_FIO7_7_bin'],
      dtype='object', length=163)

In [11]:
x_col = X_2_2.columns
y_col = 'BAD_FLAG'

In [22]:
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )
validation_generator = DataGenerator(valid_for, x_col, y_col, batch_size=valid_for.shape[0], alpha = None, class_w = None)

In [23]:
from keras.utils import np_utils

In [24]:
Y_test_2 = np_utils.to_categorical( y_test, 2) # преобразовываем в 2 класса 

In [25]:
len(x_col)

163

In [26]:
train_for.shape

(1507599, 164)

In [114]:
#n_features = 51
n_features = len(X_2_2.columns)

x = Input(shape=(  n_features, 1))
n_pool = 2


n_kernel = 7
n_filters = 6

n_strides = 1
#n_strides_pool = 2
n_strides_pool=None
classes = 2
hidden = 64 #256
drop_out = 0.25
#padding_pool = 'same'
padding_pool = 'valid'

#1
y = Conv1D(filters=n_filters, kernel_size=n_kernel, strides=n_strides, padding='same')(x)
          # kernel_regularizer = tf.keras.regularizers.l2(0.01))(x) 
y = BatchNormalization()(y)
y = ReLU()(y)
#y = Dropout(rate=drop_rate)(y)
y = MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides = n_strides_pool)(y) ##

#2
y = Conv1D(filters=n_filters*2, kernel_size=n_kernel, strides=n_strides, padding='same')(y) 
          #,kernel_regularizer = tf.keras.regularizers.l2(0.01))(y) 

y = BatchNormalization()(y)
y = ReLU()(y)
y = MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides = n_strides_pool)(y) ##

#3
y = Conv1D(filters=n_filters*(3+1), kernel_size=n_kernel, strides=n_strides, padding='same')(y) 
          #, kernel_regularizer = tf.keras.regularizers.l2(0.01))(y) 
y = BatchNormalization()(y)
y = ReLU()(y)
y = MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides = n_strides_pool)(y) ##

z = GlobalAveragePooling1D()(y)

#z = Flatten()(y)
z = Dense(hidden, activation='relu')(z)
z = Dropout(drop_out)(z)

z = Dense(hidden, activation='relu')(z)
z = Dropout(drop_out)(z)

predictions = Dense(classes, activation='softmax')(z)

In [115]:
model_cnn3 = Model(inputs=x, outputs=predictions)
model_cnn3.compile(loss='categorical_crossentropy',
                   #optimizer = tf.keras.optimizers.Ftrl(learning_rate=0.003, learning_rate_power=-0.5,  l2_regularization_strength=0.3) ,
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                   #optimizer = tf.keras.optimizers.Adamax(learning_rate=0.006) ,
                   #optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), - не уменьшался loss
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

model_cnn3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 163, 1)]          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 163, 6)            48        
_________________________________________________________________
batch_normalization (BatchNo (None, 163, 6)            24        
_________________________________________________________________
re_lu (ReLU)                 (None, 163, 6)            0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 81, 6)             0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 81, 12)            516       
_________________________________________________________________
batch_normalization_1 (Batch (None, 81, 12)            48    

In [37]:
def max_pool(min_pool, n_pool, padding_pool,  str_pool, y, str_min = 1):
    #for 3D-tensor
    if y.shape[1] < (min_pool):
        
        return MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides=str_min)(y)
    else:
        return MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides=str_pool)(y)

## CNN

In [47]:
def model_cnn_3(n_features=163, n_pool=2, n_kernel=5, n_filters=5,
                n_strides=1, n_strides_pool=None, classes=2, hidden=64, drop_out=0.25,
                l2_conv=None, reg=1E-5, reg_dense=1E-5, _bias=True, gl_pool_max=False, padding_pool='valid',
                padding_conv = 'same', n_lay=3, min_pool = 15):

    if reg == None:
        l2_batch_gamma = None
        l2_batch_betta = None
    else:
        l2_batch_gamma = l2(reg)
        l2_batch_betta = l2(reg)

    if reg_dense == None:
        kernel_regularizer = None
    else:
        kernel_regularizer = l2(reg)

    # 1 слой

    x = Input(shape=(n_features, 1))
    y = Conv1D(filters=n_filters, kernel_size=n_kernel,
               strides=n_strides, padding=padding_conv, use_bias=_bias)(x)

    y = BatchNormalization(gamma_regularizer=l2_batch_gamma,
                           beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
    y = max_pool(min_pool, n_pool, padding_pool, n_strides_pool, y)

    for i in range(1, n_lay):
        y = Conv1D(filters=(i + 1)* n_filters, kernel_size=n_kernel,
                   strides=n_strides, padding=padding_conv, use_bias=_bias)(y)
        y = BatchNormalization(gamma_regularizer=l2_batch_gamma,
                           beta_regularizer=l2_batch_betta)(y)
        y = ReLU()(y)
        y = max_pool(min_pool, n_pool, padding_pool, n_strides_pool, y)
    if gl_pool_max:
        z = GlobalMaxPooling1D()(y)
        
    else:
        z = GlobalAveragePooling1D()(y)
        
    z = Dense(hidden, activation='relu', kernel_regularizer = kernel_regularizer)(z)
    z = Dropout(drop_out)(z)
    z = Dense(hidden, activation='relu', kernel_regularizer = kernel_regularizer)(z)
    z = Dropout(drop_out)(z)
    predictions = Dense(classes, activation='softmax')(z)

    model = Model(inputs=x, outputs=predictions)


    return model

## CNN-3 test

In [43]:
model_test = model_cnn_3()

In [45]:
model_test.compile(loss='categorical_crossentropy',
                   #optimizer = tf.keras.optimizers.Ftrl(learning_rate=0.003, learning_rate_power=-0.5,  l2_regularization_strength=0.3) ,
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                   #optimizer = tf.keras.optimizers.Adamax(learning_rate=0.006) ,
                   #optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), - не уменьшался loss
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

model_test.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 163, 1)]          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 163, 5)            30        
_________________________________________________________________
batch_normalization_7 (Batch (None, 163, 5)            20        
_________________________________________________________________
re_lu_7 (ReLU)               (None, 163, 5)            0         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 81, 5)             0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 81, 10)            260       
_________________________________________________________________
batch_normalization_8 (Batch (None, 81, 10)           

## CNN-6 test

In [46]:
model_test = model_cnn_3(n_lay=6)
model_test.compile(loss='categorical_crossentropy',
                   #optimizer = tf.keras.optimizers.Ftrl(learning_rate=0.003, learning_rate_power=-0.5,  l2_regularization_strength=0.3) ,
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                   #optimizer = tf.keras.optimizers.Adamax(learning_rate=0.006) ,
                   #optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), - не уменьшался loss
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

model_test.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 163, 1)]          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 163, 5)            30        
_________________________________________________________________
batch_normalization_10 (Batc (None, 163, 5)            20        
_________________________________________________________________
re_lu_10 (ReLU)              (None, 163, 5)            0         
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 81, 5)             0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 81, 10)            260       
_________________________________________________________________
batch_normalization_11 (Batc (None, 81, 10)           

## GridSearch of hyperparameters

In [59]:
param = itertools.product(gl_p, l2_batch, n_ker, n_fil, d_hidden, drop_out, reg_dense, min_pool, n_lay)

In [56]:
i = 0
for p in param:
    print(i, p)
    i += 1

0 (False, 0.0002, 5, 5, 60, 0.25, 0.0002, 20, 3)
1 (False, 0.0002, 5, 5, 60, 0.25, 0.0002, 20, 6)
2 (False, 0.0002, 5, 5, 100, 0.25, 0.0002, 20, 3)
3 (False, 0.0002, 5, 5, 100, 0.25, 0.0002, 20, 6)
4 (False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 3)
5 (False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 6)
6 (False, 0.0002, 5, 15, 100, 0.25, 0.0002, 20, 3)
7 (False, 0.0002, 5, 15, 100, 0.25, 0.0002, 20, 6)
8 (False, 0.0002, 7, 5, 60, 0.25, 0.0002, 20, 3)
9 (False, 0.0002, 7, 5, 60, 0.25, 0.0002, 20, 6)
10 (False, 0.0002, 7, 5, 100, 0.25, 0.0002, 20, 3)
11 (False, 0.0002, 7, 5, 100, 0.25, 0.0002, 20, 6)
12 (False, 0.0002, 7, 15, 60, 0.25, 0.0002, 20, 3)
13 (False, 0.0002, 7, 15, 60, 0.25, 0.0002, 20, 6)
14 (False, 0.0002, 7, 15, 100, 0.25, 0.0002, 20, 3)
15 (False, 0.0002, 7, 15, 100, 0.25, 0.0002, 20, 6)
16 (True, 0.0002, 5, 5, 60, 0.25, 0.0002, 20, 3)
17 (True, 0.0002, 5, 5, 60, 0.25, 0.0002, 20, 6)
18 (True, 0.0002, 5, 5, 100, 0.25, 0.0002, 20, 3)
19 (True, 0.0002, 5, 5, 100, 0.25, 0.0002, 20,

In [54]:
gl_p = [False, True] #0
l2_batch = [ 0.0002] #1
n_ker = [5, 7] #2
n_fil = [ 5, 15] #3
d_hidden = [60, 100] #4
drop_out = [0.25] #5
reg_dense = [ 0.0002] #6
min_pool = [ 20] #7
n_lay = [3, 6] #8

In [57]:
inp_shape = X_2_2.shape[1]
inp_shape

163

## GridSearch and training CNN-3, CNN-6

In [62]:
VERBOSE = 2
BATCH_SIZE = 512
NB_EPOCH = 150
class_weighting = {0: 0.05, 1: 1}

result_all_8 = pd.DataFrame()
result_all_8['name_model'] = None
result_all_8['params'] = None
result_all_8['val_GINI'] = None
result_all_8['val_APS'] = None
result_all_8['test_GINI'] = None
result_all_8['test_APS'] = None
j = 0

for p in param:
    print('=====================================================================================================')
    print(inp_shape, X_2_2.shape, p)

    model_grid = model_cnn_3(reg=p[1], reg_dense=p[6], n_features=inp_shape, n_pool=2, n_kernel=p[2],
                             n_filters=p[3], n_strides=1, classes=2,
                             hidden=p[4], drop_out=p[5],
                             gl_pool_max=p[0], min_pool=p[7], n_lay=p[8])

    model_grid.compile(loss='categorical_crossentropy',
                       optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
                       metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    history_XX = model_grid.fit_generator(generator=training_generator,  # training_aug,
                                          # Y_test_2 = np_utils.to_categorical( y_val, 2)
                                          validation_data=(X_2_2, Y_test_2),
                                          epochs=NB_EPOCH, verbose=VERBOSE, class_weight=class_weighting,
                                          callbacks=[_time, EarlyStopping(monitor='val_loss', patience=20),
                                                     Metrics(valid_data=(X_2_2, Y_test_2))])

    res_model_ = pd.DataFrame(
        history_XX.history, columns=history_XX.history.keys())
    dd = str(200000 + datetime.now().month*100 +
             datetime.now().day) + '_' + str(datetime.now().hour)
    name_m = 'model_cnn_internal_grid_' + str(p[0]) + '_' + str(
        p[2]) + '_' + str(p[3]) + '_' + str(p[4]) + '_' + str(p[8]) + '_' + str(dd)

    model_grid.save(name_m + '.h5')
    res_model_.to_csv(name_m + '.csv')

    predict_class_val = model_grid.predict(X_2_2)
    APS = metrics.average_precision_score(y_test, predict_class_val[:, 1])
    GINI = 2*(metrics.roc_auc_score(y_test, predict_class_val[:, 1])) - 1

    predict_class_test = model_grid.predict(X_3_2)
    APS_t = metrics.average_precision_score( y_val, predict_class_test[:, 1])
    GINI_t = 2*(metrics.roc_auc_score( y_val, predict_class_test[:, 1])) - 1

    result_all_8.at[j, 'name_model'] = name_m
    result_all_8.at[j, 'params'] = str(p)
    result_all_8.at[j, 'val_GINI'] = GINI
    result_all_8.at[j, 'val_APS'] = APS
    result_all_8.at[j, 'test_GINI'] = GINI_t
    result_all_8.at[j, 'test_APS'] = APS_t

    result_all_8.to_csv('model_cnn_internal_grid_all.csv')
    j += 1
    print(name_m, 'valid_for_train: ', GINI, APS, '| test: ', GINI_t, APS_t)

163 (185400, 163) (False, 0.0002, 5, 5, 100, 0.25, 0.0002, 20, 3)
train_begin | time:  27.04.2021 19:27:26
Epoch 1/150
epoch_end | time:  27.04.2021 19:36:06
 — val_aps:  0.049590 — val_a: 0.895624
2945/2945 - 552s - loss: 0.0216 - accuracy: 0.9388 - auc_7: 0.9825 - precision_7: 0.9388 - recall_7: 0.9388 - val_loss: 0.0815 - val_accuracy: 0.9756 - val_auc_7: 0.9949 - val_precision_7: 0.9756 - val_recall_7: 0.9756
Epoch 2/150
epoch_end | time:  27.04.2021 19:45:54
 — val_aps:  0.056728 — val_a: 0.902807
2945/2945 - 607s - loss: 0.0146 - accuracy: 0.9669 - auc_7: 0.9899 - precision_7: 0.9669 - recall_7: 0.9669 - val_loss: 0.0224 - val_accuracy: 0.9956 - val_auc_7: 0.9989 - val_precision_7: 0.9956 - val_recall_7: 0.9956
Epoch 3/150
epoch_end | time:  27.04.2021 19:57:28
 — val_aps:  0.050298 — val_a: 0.896627
2945/2945 - 699s - loss: 0.0131 - accuracy: 0.9725 - auc_7: 0.9910 - precision_7: 0.9725 - recall_7: 0.9725 - val_loss: 0.0330 - val_accuracy: 0.9929 - val_auc_7: 0.9989 - val_precis

Epoch 4/150
epoch_end | time:  28.04.2021 00:37:11
 — val_aps:  0.059409 — val_a: 0.930134
2945/2945 - 840s - loss: 0.0069 - accuracy: 0.9873 - auc_8: 0.9980 - precision_8: 0.9873 - recall_8: 0.9873 - val_loss: 0.0671 - val_accuracy: 0.9806 - val_auc_8: 0.9983 - val_precision_8: 0.9806 - val_recall_8: 0.9806
Epoch 5/150
epoch_end | time:  28.04.2021 00:50:57
 — val_aps:  0.052820 — val_a: 0.926241
2945/2945 - 830s - loss: 0.0066 - accuracy: 0.9871 - auc_8: 0.9982 - precision_8: 0.9871 - recall_8: 0.9871 - val_loss: 0.0456 - val_accuracy: 0.9916 - val_auc_8: 0.9986 - val_precision_8: 0.9916 - val_recall_8: 0.9916
Epoch 6/150
epoch_end | time:  28.04.2021 01:05:06
 — val_aps:  0.052299 — val_a: 0.924121
2945/2945 - 847s - loss: 0.0065 - accuracy: 0.9880 - auc_8: 0.9984 - precision_8: 0.9880 - recall_8: 0.9880 - val_loss: 0.0585 - val_accuracy: 0.9821 - val_auc_8: 0.9976 - val_precision_8: 0.9821 - val_recall_8: 0.9821
Epoch 7/150
epoch_end | time:  28.04.2021 01:18:31
 — val_aps:  0.0604

train_begin | time:  28.04.2021 05:58:00
Epoch 1/150
epoch_end | time:  28.04.2021 06:18:19
 — val_aps:  0.084371 — val_a: 0.934472
2945/2945 - 1252s - loss: 0.0195 - accuracy: 0.9613 - auc_9: 0.9905 - precision_9: 0.9613 - recall_9: 0.9613 - val_loss: 0.2132 - val_accuracy: 0.9304 - val_auc_9: 0.9679 - val_precision_9: 0.9304 - val_recall_9: 0.9304
Epoch 2/150
epoch_end | time:  28.04.2021 06:39:03
 — val_aps:  0.083054 — val_a: 0.935937
2945/2945 - 1242s - loss: 0.0110 - accuracy: 0.9776 - auc_9: 0.9948 - precision_9: 0.9776 - recall_9: 0.9776 - val_loss: 0.2517 - val_accuracy: 0.9216 - val_auc_9: 0.9633 - val_precision_9: 0.9216 - val_recall_9: 0.9216
Epoch 3/150
epoch_end | time:  28.04.2021 06:59:14
 — val_aps:  0.064522 — val_a: 0.939421
2945/2945 - 1211s - loss: 0.0099 - accuracy: 0.9797 - auc_9: 0.9955 - precision_9: 0.9797 - recall_9: 0.9797 - val_loss: 0.1759 - val_accuracy: 0.9466 - val_auc_9: 0.9746 - val_precision_9: 0.9466 - val_recall_9: 0.9466
Epoch 4/150
epoch_end | ti

 — val_aps:  0.074099 — val_a: 0.943125
2945/2945 - 1266s - loss: 0.0076 - accuracy: 0.9859 - auc_9: 0.9976 - precision_9: 0.9859 - recall_9: 0.9859 - val_loss: 0.0666 - val_accuracy: 0.9938 - val_auc_9: 0.9992 - val_precision_9: 0.9938 - val_recall_9: 0.9938
Epoch 28/150
epoch_end | time:  28.04.2021 16:00:42
 — val_aps:  0.075814 — val_a: 0.941000
2945/2945 - 1163s - loss: 0.0074 - accuracy: 0.9863 - auc_9: 0.9978 - precision_9: 0.9863 - recall_9: 0.9863 - val_loss: 0.0262 - val_accuracy: 0.9937 - val_auc_9: 0.9992 - val_precision_9: 0.9937 - val_recall_9: 0.9937
Epoch 29/150
epoch_end | time:  28.04.2021 16:20:23
 — val_aps:  0.066644 — val_a: 0.945831
2945/2945 - 1176s - loss: 0.0074 - accuracy: 0.9864 - auc_9: 0.9977 - precision_9: 0.9864 - recall_9: 0.9864 - val_loss: 0.0518 - val_accuracy: 0.9856 - val_auc_9: 0.9978 - val_precision_9: 0.9856 - val_recall_9: 0.9856
Epoch 30/150
epoch_end | time:  28.04.2021 16:41:50
 — val_aps:  0.060094 — val_a: 0.935451
2945/2945 - 1307s - loss

Epoch 20/150
epoch_end | time:  29.04.2021 14:11:41
 — val_aps:  0.052432 — val_a: 0.924476
2945/2945 - 2673s - loss: 0.0043 - accuracy: 0.9894 - auc_10: 0.9992 - precision_10: 0.9894 - recall_10: 0.9894 - val_loss: 0.0221 - val_accuracy: 0.9950 - val_auc_10: 0.9988 - val_precision_10: 0.9950 - val_recall_10: 0.9950
Epoch 21/150
epoch_end | time:  29.04.2021 14:59:08
 — val_aps:  0.092263 — val_a: 0.931179
2945/2945 - 2842s - loss: 0.0042 - accuracy: 0.9896 - auc_10: 0.9992 - precision_10: 0.9896 - recall_10: 0.9896 - val_loss: 0.0192 - val_accuracy: 0.9968 - val_auc_10: 0.9983 - val_precision_10: 0.9968 - val_recall_10: 0.9968
Epoch 22/150
epoch_end | time:  29.04.2021 15:43:29
 — val_aps:  0.058704 — val_a: 0.931380
2945/2945 - 2644s - loss: 0.0041 - accuracy: 0.9898 - auc_10: 0.9992 - precision_10: 0.9898 - recall_10: 0.9898 - val_loss: 0.0212 - val_accuracy: 0.9948 - val_auc_10: 0.9988 - val_precision_10: 0.9948 - val_recall_10: 0.9948
Epoch 23/150
epoch_end | time:  29.04.2021 16:

...

Epoch 18/150
epoch_end | time:  02.05.2021 21:00:47
 — val_aps:  0.063248 — val_a: 0.933769
2945/2945 - 1659s - loss: 0.0039 - accuracy: 0.9895 - auc_18: 0.9993 - precision_18: 0.9895 - recall_18: 0.9895 - val_loss: 0.0270 - val_accuracy: 0.9898 - val_auc_18: 0.9992 - val_precision_18: 0.9898 - val_recall_18: 0.9898
Epoch 19/150
epoch_end | time:  02.05.2021 21:28:42
 — val_aps:  0.078973 — val_a: 0.923833
2945/2945 - 1676s - loss: 0.0039 - accuracy: 0.9897 - auc_18: 0.9993 - precision_18: 0.9897 - recall_18: 0.9897 - val_loss: 0.0237 - val_accuracy: 0.9927 - val_auc_18: 0.9986 - val_precision_18: 0.9927 - val_recall_18: 0.9927
Epoch 20/150
epoch_end | time:  02.05.2021 21:56:24
 — val_aps:  0.057505 — val_a: 0.933805
2945/2945 - 1659s - loss: 0.0038 - accuracy: 0.9898 - auc_18: 0.9993 - precision_18: 0.9898 - recall_18: 0.9898 - val_loss: 0.0349 - val_accuracy: 0.9900 - val_auc_18: 0.9986 - val_precision_18: 0.9900 - val_recall_18: 0.9900
Epoch 21/150
epoch_end | time:  02.05.2021 22:

Epoch 17/150
epoch_end | time:  03.05.2021 05:22:48
 — val_aps:  0.073454 — val_a: 0.932753
2945/2945 - 1004s - loss: 0.0070 - accuracy: 0.9866 - auc_19: 0.9980 - precision_19: 0.9866 - recall_19: 0.9866 - val_loss: 0.0299 - val_accuracy: 0.9930 - val_auc_19: 0.9993 - val_precision_19: 0.9930 - val_recall_19: 0.9930
Epoch 18/150
epoch_end | time:  03.05.2021 05:39:36
 — val_aps:  0.073887 — val_a: 0.939629
2945/2945 - 1007s - loss: 0.0071 - accuracy: 0.9864 - auc_19: 0.9978 - precision_19: 0.9864 - recall_19: 0.9864 - val_loss: 0.0810 - val_accuracy: 0.9732 - val_auc_19: 0.9956 - val_precision_19: 0.9732 - val_recall_19: 0.9732
Epoch 19/150
epoch_end | time:  03.05.2021 05:56:23
 — val_aps:  0.082392 — val_a: 0.938052
2945/2945 - 1006s - loss: 0.0070 - accuracy: 0.9866 - auc_19: 0.9980 - precision_19: 0.9866 - recall_19: 0.9866 - val_loss: 0.0171 - val_accuracy: 0.9968 - val_auc_19: 0.9991 - val_precision_19: 0.9968 - val_recall_19: 0.9968
Epoch 20/150
epoch_end | time:  03.05.2021 06:

Epoch 3/150
epoch_end | time:  03.05.2021 12:46:13
 — val_aps:  0.072081 — val_a: 0.948574
2945/2945 - 1481s - loss: 0.0062 - accuracy: 0.9878 - auc_20: 0.9984 - precision_20: 0.9878 - recall_20: 0.9878 - val_loss: 0.0478 - val_accuracy: 0.9804 - val_auc_20: 0.9986 - val_precision_20: 0.9804 - val_recall_20: 0.9804
Epoch 4/150
epoch_end | time:  03.05.2021 13:10:59
 — val_aps:  0.055631 — val_a: 0.939937
2945/2945 - 1486s - loss: 0.0057 - accuracy: 0.9884 - auc_20: 0.9987 - precision_20: 0.9884 - recall_20: 0.9884 - val_loss: 0.0541 - val_accuracy: 0.9835 - val_auc_20: 0.9974 - val_precision_20: 0.9835 - val_recall_20: 0.9835
Epoch 5/150
epoch_end | time:  03.05.2021 13:35:50
 — val_aps:  0.073445 — val_a: 0.954491
2945/2945 - 1489s - loss: 0.0055 - accuracy: 0.9883 - auc_20: 0.9988 - precision_20: 0.9883 - recall_20: 0.9883 - val_loss: 0.0554 - val_accuracy: 0.9902 - val_auc_20: 0.9991 - val_precision_20: 0.9902 - val_recall_20: 0.9902
Epoch 6/150
epoch_end | time:  03.05.2021 14:00:3

Epoch 29/150
epoch_end | time:  03.05.2021 23:43:15
 — val_aps:  0.058607 — val_a: 0.916965
2945/2945 - 1516s - loss: 0.0034 - accuracy: 0.9911 - auc_20: 0.9994 - precision_20: 0.9911 - recall_20: 0.9911 - val_loss: 0.0441 - val_accuracy: 0.9842 - val_auc_20: 0.9982 - val_precision_20: 0.9842 - val_recall_20: 0.9842
Epoch 30/150
epoch_end | time:  04.05.2021 00:08:26
 — val_aps:  0.045162 — val_a: 0.901276
2945/2945 - 1510s - loss: 0.0034 - accuracy: 0.9914 - auc_20: 0.9994 - precision_20: 0.9914 - recall_20: 0.9914 - val_loss: 0.0302 - val_accuracy: 0.9932 - val_auc_20: 0.9980 - val_precision_20: 0.9932 - val_recall_20: 0.9932
Epoch 31/150
epoch_end | time:  04.05.2021 00:33:37
 — val_aps:  0.063009 — val_a: 0.925744
2945/2945 - 1510s - loss: 0.0034 - accuracy: 0.9914 - auc_20: 0.9994 - precision_20: 0.9914 - recall_20: 0.9914 - val_loss: 0.0304 - val_accuracy: 0.9900 - val_auc_20: 0.9988 - val_precision_20: 0.9900 - val_recall_20: 0.9900
Epoch 32/150
epoch_end | time:  04.05.2021 00:

NameError: name 'GlobalMaxPooling1D' is not defined

## GridSearch and training CNN-6, CNN-8

In [64]:
param = ((False, 0.0001, 5, 5, 30, 0.25, 0.0002, 20, 6), 
         (False, 0.0001, 5, 5, 30, 0.25, 0.0002, 20, 8),
        (False, 0.0001, 5, 10, 30, 0.25, 0.0002, 20, 6),
         (False, 0.0001, 5, 10, 30, 0.25, 0.0002, 20, 8),
        (False, 0.0001, 3, 10, 100, 0.25, 0.0002, 20, 6),
         (False, 0.0001, 3, 10, 100, 0.25, 0.0002, 20, 8),
         (False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 6) ,
        (False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 8))

In [65]:
VERBOSE = 2
BATCH_SIZE = 512
NB_EPOCH = 150
class_weighting = {0: 0.05, 1: 1}

result_all_8 = pd.DataFrame()
result_all_8['name_model'] = None
result_all_8['params'] = None
result_all_8['val_GINI'] = None
result_all_8['val_APS'] = None
result_all_8['test_GINI'] = None
result_all_8['test_APS'] = None
j = 0

for p in param:
    print('=====================================================================================================')
    print(inp_shape, X_2_2.shape, p)

    model_grid = model_cnn_3(reg=p[1], reg_dense=p[6], n_features=inp_shape, n_pool=2, n_kernel=p[2],
                             n_filters=p[3], n_strides=1, classes=2,
                             hidden=p[4], drop_out=p[5],
                             gl_pool_max=p[0], min_pool=p[7], n_lay=p[8])

    model_grid.compile(loss='categorical_crossentropy',
                       optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
                       metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    history_XX = model_grid.fit_generator(generator=training_generator,  # training_aug,
                                          # Y_test_2 = np_utils.to_categorical( y_val, 2)
                                          validation_data=(X_2_2, Y_test_2),
                                          epochs=NB_EPOCH, verbose=VERBOSE, class_weight=class_weighting,
                                          callbacks=[_time, EarlyStopping(monitor='val_loss', patience=20),
                                                     Metrics(valid_data=(X_2_2, Y_test_2))])

    res_model_ = pd.DataFrame(
        history_XX.history, columns=history_XX.history.keys())
    dd = str(200000 + datetime.now().month*100 +
             datetime.now().day) + '_' + str(datetime.now().hour)
    name_m = 'model_cnn_internal_grid_' + str(p[0]) + '_' + str(
        p[2]) + '_' + str(p[3]) + '_' + str(p[4]) + '_' + str(p[8]) + '_' + str(dd)

    model_grid.save(name_m + '.h5')
    res_model_.to_csv(name_m + '.csv')

    predict_class_val = model_grid.predict(X_2_2)
    APS = metrics.average_precision_score(y_test, predict_class_val[:, 1])
    GINI = 2*(metrics.roc_auc_score(y_test, predict_class_val[:, 1])) - 1

    predict_class_test = model_grid.predict(X_3_2)
    APS_t = metrics.average_precision_score( y_val, predict_class_test[:, 1])
    GINI_t = 2*(metrics.roc_auc_score( y_val, predict_class_test[:, 1])) - 1

    result_all_8.at[j, 'name_model'] = name_m
    result_all_8.at[j, 'params'] = str(p)
    result_all_8.at[j, 'val_GINI'] = GINI
    result_all_8.at[j, 'val_APS'] = APS
    result_all_8.at[j, 'test_GINI'] = GINI_t
    result_all_8.at[j, 'test_APS'] = APS_t

    result_all_8.to_csv('model_cnn_internal_grid_all_x8.csv')
    j += 1
    print(name_m, 'valid_for_train: ', GINI, APS, '| test: ', GINI_t, APS_t)

163 (185400, 163) (False, 0.0001, 5, 5, 30, 0.25, 0.0002, 20, 6)
train_begin | time:  04.05.2021 07:57:41
Epoch 1/150
epoch_end | time:  04.05.2021 08:05:55
 — val_aps:  0.033276 — val_a: 0.906364
2945/2945 - 509s - loss: 0.0142 - accuracy: 0.9747 - auc_21: 0.9947 - precision_21: 0.9747 - recall_21: 0.9747 - val_loss: 0.1407 - val_accuracy: 0.9572 - val_auc_21: 0.9870 - val_precision_21: 0.9572 - val_recall_21: 0.9572
Epoch 2/150
epoch_end | time:  04.05.2021 08:14:22
 — val_aps:  0.055854 — val_a: 0.927169
2945/2945 - 506s - loss: 0.0080 - accuracy: 0.9847 - auc_21: 0.9973 - precision_21: 0.9847 - recall_21: 0.9847 - val_loss: 0.0529 - val_accuracy: 0.9870 - val_auc_21: 0.9966 - val_precision_21: 0.9870 - val_recall_21: 0.9870
Epoch 3/150
epoch_end | time:  04.05.2021 08:22:40
 — val_aps:  0.064485 — val_a: 0.933646
2945/2945 - 498s - loss: 0.0071 - accuracy: 0.9859 - auc_21: 0.9979 - precision_21: 0.9859 - recall_21: 0.9859 - val_loss: 0.0444 - val_accuracy: 0.9873 - val_auc_21: 0.99

Epoch 27/150
epoch_end | time:  04.05.2021 11:46:44
 — val_aps:  0.059339 — val_a: 0.936276
2945/2945 - 505s - loss: 0.0049 - accuracy: 0.9866 - auc_21: 0.9990 - precision_21: 0.9866 - recall_21: 0.9866 - val_loss: 0.0330 - val_accuracy: 0.9926 - val_auc_21: 0.9988 - val_precision_21: 0.9926 - val_recall_21: 0.9926
Epoch 28/150
epoch_end | time:  04.05.2021 11:55:22
 — val_aps:  0.047969 — val_a: 0.925400
2945/2945 - 518s - loss: 0.0048 - accuracy: 0.9868 - auc_21: 0.9990 - precision_21: 0.9868 - recall_21: 0.9868 - val_loss: 0.0586 - val_accuracy: 0.9749 - val_auc_21: 0.9972 - val_precision_21: 0.9749 - val_recall_21: 0.9749
Epoch 29/150
epoch_end | time:  04.05.2021 12:04:26
 — val_aps:  0.072918 — val_a: 0.937158
2945/2945 - 546s - loss: 0.0048 - accuracy: 0.9865 - auc_21: 0.9990 - precision_21: 0.9865 - recall_21: 0.9865 - val_loss: 0.0189 - val_accuracy: 0.9960 - val_auc_21: 0.9987 - val_precision_21: 0.9960 - val_recall_21: 0.9960
Epoch 30/150
epoch_end | time:  04.05.2021 12:13:

Epoch 7/150
epoch_end | time:  04.05.2021 16:02:37
 — val_aps:  0.072346 — val_a: 0.931688
2945/2945 - 760s - loss: 0.0057 - accuracy: 0.9883 - auc_22: 0.9986 - precision_22: 0.9883 - recall_22: 0.9883 - val_loss: 0.0580 - val_accuracy: 0.9847 - val_auc_22: 0.9981 - val_precision_22: 0.9847 - val_recall_22: 0.9847
Epoch 8/150
epoch_end | time:  04.05.2021 16:15:08
 — val_aps:  0.063050 — val_a: 0.931426
2945/2945 - 757s - loss: 0.0055 - accuracy: 0.9885 - auc_22: 0.9987 - precision_22: 0.9885 - recall_22: 0.9885 - val_loss: 0.0229 - val_accuracy: 0.9945 - val_auc_22: 0.9990 - val_precision_22: 0.9945 - val_recall_22: 0.9945
Epoch 9/150
epoch_end | time:  04.05.2021 16:30:46
 — val_aps:  0.074373 — val_a: 0.929995
2945/2945 - 924s - loss: 0.0054 - accuracy: 0.9883 - auc_22: 0.9988 - precision_22: 0.9883 - recall_22: 0.9883 - val_loss: 0.0181 - val_accuracy: 0.9966 - val_auc_22: 0.9988 - val_precision_22: 0.9966 - val_recall_22: 0.9966
Epoch 10/150
epoch_end | time:  04.05.2021 16:44:29


Epoch 33/150
epoch_end | time:  04.05.2021 22:34:52
 — val_aps:  0.070155 — val_a: 0.927916
2945/2945 - 956s - loss: 0.0043 - accuracy: 0.9890 - auc_22: 0.9991 - precision_22: 0.9890 - recall_22: 0.9890 - val_loss: 0.0259 - val_accuracy: 0.9922 - val_auc_22: 0.9989 - val_precision_22: 0.9922 - val_recall_22: 0.9922
Epoch 34/150
epoch_end | time:  04.05.2021 22:48:19
 — val_aps:  0.062840 — val_a: 0.917502
2945/2945 - 799s - loss: 0.0044 - accuracy: 0.9888 - auc_22: 0.9991 - precision_22: 0.9888 - recall_22: 0.9888 - val_loss: 0.0252 - val_accuracy: 0.9947 - val_auc_22: 0.9992 - val_precision_22: 0.9947 - val_recall_22: 0.9947
Epoch 35/150
epoch_end | time:  04.05.2021 23:01:35
 — val_aps:  0.067740 — val_a: 0.927108
2945/2945 - 800s - loss: 0.0043 - accuracy: 0.9888 - auc_22: 0.9992 - precision_22: 0.9888 - recall_22: 0.9888 - val_loss: 0.0211 - val_accuracy: 0.9948 - val_auc_22: 0.9989 - val_precision_22: 0.9948 - val_recall_22: 0.9948
Epoch 36/150
epoch_end | time:  04.05.2021 23:15:

Epoch 17/150
epoch_end | time:  05.05.2021 06:49:41
 — val_aps:  0.066135 — val_a: 0.926325
2945/2945 - 1224s - loss: 0.0042 - accuracy: 0.9889 - auc_23: 0.9992 - precision_23: 0.9889 - recall_23: 0.9889 - val_loss: 0.0244 - val_accuracy: 0.9933 - val_auc_23: 0.9992 - val_precision_23: 0.9933 - val_recall_23: 0.9933
Epoch 18/150
epoch_end | time:  05.05.2021 07:10:05
 — val_aps:  0.035072 — val_a: 0.910761
2945/2945 - 1223s - loss: 0.0041 - accuracy: 0.9890 - auc_23: 0.9992 - precision_23: 0.9890 - recall_23: 0.9890 - val_loss: 0.4613 - val_accuracy: 0.8672 - val_auc_23: 0.9338 - val_precision_23: 0.8672 - val_recall_23: 0.8672
Epoch 19/150
epoch_end | time:  05.05.2021 07:30:26
 — val_aps:  0.070489 — val_a: 0.937988
2945/2945 - 1219s - loss: 0.0040 - accuracy: 0.9892 - auc_23: 0.9992 - precision_23: 0.9892 - recall_23: 0.9892 - val_loss: 0.0721 - val_accuracy: 0.9635 - val_auc_23: 0.9968 - val_precision_23: 0.9635 - val_recall_23: 0.9635
Epoch 20/150
epoch_end | time:  05.05.2021 07:

...

Epoch 9/150
epoch_end | time:  06.05.2021 14:07:52
 — val_aps:  0.067422 — val_a: 0.942789
2945/2945 - 1070s - loss: 0.0052 - accuracy: 0.9882 - auc_25: 0.9989 - precision_25: 0.9882 - recall_25: 0.9882 - val_loss: 0.0209 - val_accuracy: 0.9948 - val_auc_25: 0.9992 - val_precision_25: 0.9948 - val_recall_25: 0.9948
Epoch 10/150
epoch_end | time:  06.05.2021 14:24:40
 — val_aps:  0.059876 — val_a: 0.934734
2945/2945 - 1005s - loss: 0.0051 - accuracy: 0.9882 - auc_25: 0.9989 - precision_25: 0.9882 - recall_25: 0.9882 - val_loss: 0.0194 - val_accuracy: 0.9959 - val_auc_25: 0.9990 - val_precision_25: 0.9959 - val_recall_25: 0.9959
Epoch 11/150
epoch_end | time:  06.05.2021 14:42:28
 — val_aps:  0.076336 — val_a: 0.937261
2945/2945 - 1091s - loss: 0.0052 - accuracy: 0.9884 - auc_25: 0.9989 - precision_25: 0.9884 - recall_25: 0.9884 - val_loss: 0.0246 - val_accuracy: 0.9921 - val_auc_25: 0.9992 - val_precision_25: 0.9921 - val_recall_25: 0.9921
Epoch 12/150
epoch_end | time:  06.05.2021 15:0

model_cnn_internal_grid_False_3_10_100_6_200506_21 valid_for_train:  0.8441961479425308 0.0600593764979524 | test:  0.8304321690023344 0.05447323072032394
163 (185400, 163) (False, 0.0001, 3, 10, 100, 0.25, 0.0002, 20, 8)
train_begin | time:  06.05.2021 21:20:38
Epoch 1/150
epoch_end | time:  06.05.2021 21:41:39
 — val_aps:  0.070926 — val_a: 0.934441
2945/2945 - 1293s - loss: 0.0168 - accuracy: 0.9800 - auc_26: 0.9955 - precision_26: 0.9800 - recall_26: 0.9800 - val_loss: 0.0320 - val_accuracy: 0.9945 - val_auc_26: 0.9994 - val_precision_26: 0.9945 - val_recall_26: 0.9945
Epoch 2/150
epoch_end | time:  06.05.2021 22:01:48
 — val_aps:  0.047986 — val_a: 0.916054
2945/2945 - 1203s - loss: 0.0072 - accuracy: 0.9872 - auc_26: 0.9976 - precision_26: 0.9872 - recall_26: 0.9872 - val_loss: 0.0195 - val_accuracy: 0.9967 - val_auc_26: 0.9989 - val_precision_26: 0.9967 - val_recall_26: 0.9967
Epoch 3/150
epoch_end | time:  06.05.2021 22:23:01
 — val_aps:  0.053656 — val_a: 0.930436
2945/2945 - 

Epoch 26/150
epoch_end | time:  07.05.2021 05:26:18
 — val_aps:  0.061535 — val_a: 0.932138
2945/2945 - 1084s - loss: 0.0042 - accuracy: 0.9893 - auc_26: 0.9992 - precision_26: 0.9893 - recall_26: 0.9893 - val_loss: 0.0299 - val_accuracy: 0.9899 - val_auc_26: 0.9988 - val_precision_26: 0.9899 - val_recall_26: 0.9899
Epoch 27/150
epoch_end | time:  07.05.2021 05:44:26
 — val_aps:  0.059984 — val_a: 0.915624
2945/2945 - 1088s - loss: 0.0041 - accuracy: 0.9894 - auc_26: 0.9992 - precision_26: 0.9894 - recall_26: 0.9894 - val_loss: 0.0355 - val_accuracy: 0.9912 - val_auc_26: 0.9990 - val_precision_26: 0.9912 - val_recall_26: 0.9912
Epoch 28/150
epoch_end | time:  07.05.2021 06:02:22
 — val_aps:  0.067143 — val_a: 0.931096
2945/2945 - 1075s - loss: 0.0041 - accuracy: 0.9893 - auc_26: 0.9992 - precision_26: 0.9893 - recall_26: 0.9893 - val_loss: 0.0457 - val_accuracy: 0.9846 - val_auc_26: 0.9983 - val_precision_26: 0.9846 - val_recall_26: 0.9846
Epoch 29/150
epoch_end | time:  07.05.2021 06:

Epoch 17/150
epoch_end | time:  07.05.2021 15:38:51
 — val_aps:  0.057235 — val_a: 0.924333
2945/2945 - 1840s - loss: 0.0044 - accuracy: 0.9895 - auc_27: 0.9992 - precision_27: 0.9895 - recall_27: 0.9895 - val_loss: 0.0332 - val_accuracy: 0.9896 - val_auc_27: 0.9989 - val_precision_27: 0.9896 - val_recall_27: 0.9896
Epoch 18/150
epoch_end | time:  07.05.2021 16:09:13
 — val_aps:  0.057152 — val_a: 0.920704
2945/2945 - 1805s - loss: 0.0043 - accuracy: 0.9892 - auc_27: 0.9992 - precision_27: 0.9892 - recall_27: 0.9892 - val_loss: 0.0346 - val_accuracy: 0.9871 - val_auc_27: 0.9986 - val_precision_27: 0.9871 - val_recall_27: 0.9871
Epoch 19/150
epoch_end | time:  07.05.2021 16:38:55
 — val_aps:  0.068711 — val_a: 0.927643
2945/2945 - 1811s - loss: 0.0042 - accuracy: 0.9895 - auc_27: 0.9992 - precision_27: 0.9895 - recall_27: 0.9895 - val_loss: 0.0200 - val_accuracy: 0.9958 - val_auc_27: 0.9989 - val_precision_27: 0.9958 - val_recall_27: 0.9958
Epoch 20/150
epoch_end | time:  07.05.2021 17:

Epoch 17/150
epoch_end | time:  08.05.2021 06:32:11
 — val_aps:  0.061163 — val_a: 0.929989
2945/2945 - 2220s - loss: 0.0043 - accuracy: 0.9890 - auc_28: 0.9992 - precision_28: 0.9890 - recall_28: 0.9890 - val_loss: 0.0684 - val_accuracy: 0.9728 - val_auc_28: 0.9966 - val_precision_28: 0.9728 - val_recall_28: 0.9728
Epoch 18/150
epoch_end | time:  08.05.2021 07:08:34
 — val_aps:  0.074984 — val_a: 0.926765
2945/2945 - 2178s - loss: 0.0043 - accuracy: 0.9888 - auc_28: 0.9992 - precision_28: 0.9888 - recall_28: 0.9888 - val_loss: 0.0566 - val_accuracy: 0.9846 - val_auc_28: 0.9984 - val_precision_28: 0.9846 - val_recall_28: 0.9846
Epoch 19/150
epoch_end | time:  08.05.2021 07:45:34
 — val_aps:  0.069554 — val_a: 0.929757
2945/2945 - 2241s - loss: 0.0042 - accuracy: 0.9893 - auc_28: 0.9993 - precision_28: 0.9893 - recall_28: 0.9893 - val_loss: 0.0192 - val_accuracy: 0.9965 - val_auc_28: 0.9988 - val_precision_28: 0.9965 - val_recall_28: 0.9965
Epoch 20/150
epoch_end | time:  08.05.2021 08: